In [1]:
import os
import h5py
import numpy as np

input_datafile = h5py.File('../input/uploads/uploads/tryp')

train_norm_images = np.array(input_datafile['train_norm'])
train_trypo_images = np.array(input_datafile['train_trypo'])
valid_norm_images = np.array(input_datafile['valid_norm'])
valid_trypo_images = np.array(input_datafile['valid_trypo'])

In [2]:
import matplotlib.pyplot as plt
from random import shuffle
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils
    
def process_images(image):
    return np.divide(image, 255)

def permutate_data(norm, trypo):
    print(norm.shape)
  #  norm = process_images(np.array(norm))
   # trypo = process_images(np.array(trypo))
    X = np.concatenate([norm, trypo])
    Y1 = np.zeros(len(norm))
    Y2 = np.ones(len(trypo))
    Y = np.concatenate([Y1, Y2])
    ind = [i for i in range(len(Y))]
    shuffle(ind)
    X_new  = X[ind, :,:,:]
    Y_new = np_utils.to_categorical(Y[ind]) 
    print(Y_new.shape)
    print(X_new.shape)
    return X_new, Y_new

X, Y = permutate_data(train_norm_images, train_trypo_images)
X_val, Y_val = permutate_data(valid_norm_images, valid_trypo_images)

print(X.shape)

Using TensorFlow backend.


(10068, 224, 224, 3)
(15884, 2)
(15884, 224, 224, 3)
(500, 224, 224, 3)
(1000, 2)
(1000, 224, 224, 3)
(15884, 224, 224, 3)


In [5]:
import keras
from keras.models import Model, Sequential
from keras.layers import GlobalAveragePooling2D, Dense
from keras.applications.xception import Xception

model = Xception(include_top = False, weights='imagenet', input_shape =(224, 224, 3))
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2, activation = 'softmax')(x)
model = Model(model.input, x)
model.summary()

82477056/83683744 [============================>.] - ETA: 0s____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 111, 111, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 111, 111, 32)  128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 111, 111, 32)  0           block1_conv1_bn[0][0]            
_______________________________

In [6]:
for layer in model.layers[:-1]:
    layer.trainable = False
model.compile(optimizer = "adam", loss = "categorical_crossentropy",
             metrics = ["accuracy"])
model.evaluate(X_val, Y_val)

1000/1000 [==============================] - 14s    


[3.063140365600586, 0.59399999999999997]

In [7]:
model.fit(X, Y, epochs = 10, validation_data=[X_val, Y_val])

Train on 15884 samples, validate on 1000 samples
Epoch 1/10
15884/15884 [==============================] - 199s - loss: 0.3253 - acc: 0.8730 - val_loss: 0.2848 - val_acc: 0.8840
Epoch 2/10
15884/15884 [==============================] - 199s - loss: 0.2598 - acc: 0.8975 - val_loss: 0.2736 - val_acc: 0.8870
Epoch 3/10
15884/15884 [==============================] - 199s - loss: 0.2492 - acc: 0.9013 - val_loss: 0.2213 - val_acc: 0.9130
Epoch 4/10
15884/15884 [==============================] - 199s - loss: 0.2433 - acc: 0.9036 - val_loss: 0.2107 - val_acc: 0.9210
Epoch 5/10
15884/15884 [==============================] - 199s - loss: 0.2381 - acc: 0.9063 - val_loss: 0.2677 - val_acc: 0.8940
Epoch 6/10
15884/15884 [==============================] - 199s - loss: 0.2259 - acc: 0.9132 - val_loss: 0.2324 - val_acc: 0.9080
Epoch 7/10
15884/15884 [==============================] - 199s - loss: 0.2263 - acc: 0.9105 - val_loss: 0.2492 - val_acc: 0.9010
Epoch 8/10
15884/15884 [========================

In [ ]:
model.save_weights("vgg_0.1_weights")